In [52]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
import math
import torch
from torch import nn
import numpy as np
import pandas as pd
import argparse
import time
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
import sys
import tabulate

In [54]:
sys.path.append("./simplex/")
sys.path.append("./simplex/models/")
sys.path.append("./experiments/vgg-cifar10/")

import utils
# from vgg_noBN import VGG16, VGG16Simplex
from simplex_models import SimplexNet, Simplex
from simplex.models.lenet5 import Lenet5Simplex, Lenet5
from criterion import *
from datasets import get_dataset
from resnet import Resnet18Simplex
from plot_utils import *
from simplex.surfaces import get_basis#, compute_loss_surface

In [55]:
# arguments
n_connector = 3
n_mode = 4
LMBD = 0.1
poison_factor = 0
batch_size = 64

data_path = "/home/govind/datasets/SVHN"
model_dir = "complex/good_mode_good_con"

# reproducibility
torch.manual_seed(1)
np.random.seed(1)

In [63]:
sim_model = Lenet5Simplex
base_model = Lenet5()

In [57]:
trainloader, testloader = get_dataset(name="SVHN", data_path=data_path, batch_size=batch_size, poison_factor=poison_factor)

Using downloaded and verified file: /home/govind/datasets/SVHN/train_32x32.mat
Using downloaded and verified file: /home/govind/datasets/SVHN/extra_32x32.mat
Using downloaded and verified file: /home/govind/datasets/SVHN/test_32x32.mat


In [ ]:
simple

In [64]:
reg_pars = [0.]
for ii in range(4, n_connector + n_mode + 2):
    fix_pts = [True]*(ii)
    start_vert = len(fix_pts)
    out_dim = 10
    simplex_model = SimplexNet(out_dim, sim_model, n_vert=start_vert, fix_points=fix_pts)
    simplex_model = simplex_model.cuda()
    log_vol = (simplex_model.total_volume() + 1e-4).log()
    reg_pars.append(max(float(LMBD)/log_vol, 1e-8))

fix_pts = [True] * n_mode
n_vert = len(fix_pts)
complex_ = {ii:[ii] for ii in range(n_mode)}
simplex_model = SimplexNet(10, sim_model, n_vert=n_vert, simplicial_complex=complex_, fix_points=fix_pts).cuda()

TypeError: super(type, obj): obj must be an instance or subtype of type

In [65]:
for vv in range(n_connector):
    simplex_model.add_vert(to_simplexes=[ii for ii in range(n_mode)])

TypeError: super(type, obj): obj must be an instance or subtype of type

In [ ]:
print("Load simplical complex model")
fname = os.path.join("experiments/vgg-cifar10/saved-outputs", model_dir)
# path = sorted(glob.glob(os.path.join(fname, "*.pt")))
simplex_model.load_multiple_model(fname, frustration=True)

In [66]:
simplex_model.simplex_param_vectors.shape

torch.Size([7, 62006])

In [51]:
surf_runner(simplex_model, 
            sim_model, 
            0, 1, 4,
            trainloader, None, None, None)

TypeError: super(type, obj): obj must be an instance or subtype of type

In [29]:
v1, v2 = get_basis(simplex_model, 0, 1, 4)

In [30]:
par_vecs = simplex_model.par_vectors()

In [31]:
w0, theta0 = par_vecs[0], par_vecs[4]

In [32]:
vec = w0 - theta0

In [33]:
base_model = simplex_model.architecture(simplex_model.n_output, **simplex_model.architecture_kwargs).cuda()

In [35]:
center_pars = par_vecs[[0, 1, 4], :].mean(0).unsqueeze(0)
#center_pars = par_vecs[:1]
utils.assign_pars(center_pars, base_model)

anchor_pars = torch.cat((par_vecs.shape[0] * [center_pars]))
anchor_diffs = (par_vecs - anchor_pars)

diff_v1_projs = anchor_diffs.matmul(v1)
diff_v2_projs = anchor_diffs.matmul(v2)

range_x = 1.5*(diff_v1_projs).abs().max()# + 1
range_y = 1.5*(diff_v2_projs).abs().max()# + 

#range_x = (diff_v1_projs).abs().max() + 1
#range_y = (diff_v2_projs).abs().max() +1
#range_ = 100* range_



In [43]:
X, Y, correct, surf = compute_loss_surface(base_model, trainloader, 
                              v1, v2, proj_x = diff_v1_projs, proj_y=diff_v2_projs, loss = None,
                              coeffs_t = simplex_model.vertex_weights(),
                             range_x = range_x, range_y = range_y, n_pts=20)

AttributeError: 'Conv2d' object has no attribute 'weight_1'

In [42]:
base_model.conv1.w

AttributeError: 'Conv2d' object has no attribute 'weights'